#### LIBRERIAS

In [1]:
import pandas as pd
from datetime import datetime
from datetime import date
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import unicodedata
import math
from itertools import cycle
import xlsxwriter
import pyodbc
from datetime import timedelta
import matplotlib
import random
from IPython.display import display, HTML
import sys
import tkinter as tk
from tkinter import ttk
from tkinter import PhotoImage
from PIL import ImageTk, Image
import os
pd.options.display.float_format = "{:,.0f}".format

#### FUNCIONES

In [2]:
def calcula_dv(rut_base):
    # CONVERTIR EL RUT BASE A UNA CADENA Y REVERTIRLA
    reversed_digits = list(map(int, str(rut_base)[::-1]))
    # DEFINIR LOS FACTORES (2..7) QUE SE REPITEN EN CICLO
    factors = list(range(2, 8))
    # CALCULAR LA SUMA DE LOS PRODUCTO
    suma = sum(d * f for d, f in zip(reversed_digits, factors * (len(reversed_digits) // len(factors) + 1)))
    # CALCULA DV
    dv = (-suma) % 11
    return 'K' if dv == 10 else str(dv)

def remove_leading_zeros(rut):
    rut_part, dv = rut.split('-')
    rut_part = rut_part.lstrip('0')
    if not rut_part:
        rut_part = '0'
    return f"{rut_part}-{dv}"

def reemplazar_k(identificador):
    return identificador[:-1:] + identificador[-1].upper()

def separar_rut(numero):
    numero_str = str(numero)
    # TOMAR EL ULTIMO DIGITO
    parte_numerica = numero_str[:-1]
    digito_verificador = numero_str[-1]
    # FORMTATEAR GUION
    rut_formateado = f"{parte_numerica}-{digito_verificador}"
    return rut_formateado

#### MUESTRA CLIENTES: 

Se toma muestra clientes de Ficha PEP desde Regcheq para desde esta base tomar las operaciones y aportes y rescates.

In [3]:
df_muestra2 = pd.read_excel("Monitoreo Noviembre/Clientes PEP/muestra PEP NOVIEMBRE.xlsx")

In [4]:
identificadores = df_muestra2['IDENTIFICADOR'].tolist()
# SEPARAR POR COMAS
m = len(identificadores)
identificadores_str = ','.join([f"'{id}'" for id in identificadores])

#### CONSULTAS:
1. query_APORET: toma los aportes y los retiros
2. query_INFO: toma la info personal
3. query_CUSTODIA: operaciones del cliente

4. query_OPERACIONES ## criterio aportes e inmovilizado sin ningun tipo de operacion
5. query_SALDO_CAJA ## saldo caja primer mes y compararlo a finales, si en ambos esta sobre 10kk sospecha

In [5]:
m = df_muestra2['IDENTIFICADOR'].nunique()

In [6]:
F_i = '2023-01-01'
F = '2024-11-30' 
F_ic = '2024-11-30'
F_ix = '2024-11-01'

In [7]:
query_universal = {

    'query_operaciones': f"""

            SELECT 
                [NUM_CUENTA],
                [IDENTIFICADOR],
                [NOMBRE_CLIENTE],
                [NOMBRE_ASESOR],
                [FECHA_OPERACION],
                [COD_TIPO_OPERACION],
                [DSC_OPERACION_CONCEPTO],
                [COD_MONEDA_OP],
                [NEMOTECNICO],
                [DSC_INSTRUMENTO],
                [COD_SUB_CLASE_INSTRUMENTO],
                [CANTIDAD],
                [MONTO],
                [MONTO_OPERACION]
            FROM [Capitaria].[dbo].[OPERACION_CLIENTE_JGG]
            WHERE [IDENTIFICADOR] IN ({identificadores_str})
            AND FECHA_OPERACION BETWEEN CONVERT(datetime, '{F_i}', 120) AND CONVERT(datetime, '{F}', 120)
    """
}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_operaciones = pd.read_sql(query_universal['query_operaciones'], conn)
# CLOSE
conn.close()

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_11384\85637969.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_operaciones = pd.read_sql(query_universal['query_operaciones'], conn)


In [8]:
query_universal = {
    
        'query_APORET': f"""
        SELECT 
        CARGO_ABONO,
        NUM_CUENTA,	
        IDENTIFICADOR,	
        NOMBRE_CLI,	
        COD_MOV,	
        DSC_MOV_CAJA,	
        FECHA_MOVIMIENTO,	
        FECHA_LIQUIDACION,
        MONTO,
        NOMBRE_ASESOR,	
        TIPO_CAJA,	
        COD_MONEDA,		
        OBS_MOV_CAJA,	
        T_C,	
        MONTO_CLP

        FROM [Capitaria].[dbo].[MOV_CAJA_CLI_JGG]
        WHERE COD_MOV IN (
            'APO_PAT',
            'APO_PAT_AT',
            'APO_PAT_BT',
            'APO_PAT_LI',
            'APO_PAT_RC',
            'RET_PAT',
            'RET_PAT_BA',
            'RET_PAT_BT',
            'RET_PAT_EX',
            'RET_PAT_LI'
            )
        AND FECHA_MOVIMIENTO BETWEEN CONVERT(datetime, '{F_i}', 120) AND CONVERT(datetime, '{F}', 120)
        AND IDENTIFICADOR IN ({identificadores_str});
        """,

        'query_DATOS': f"""
        SELECT [NUM_CUENTA]
              ,[DSC_CUENTA]
              ,[ESTADO_CUENTA]
              ,[COD_MONEDA]
              ,[FECHA_OPERATIVA]
              ,[NOMBRE_CLIENTE]
              ,[NOMBRE_ASESOR]
              ,[DSC_PERFIL_RIESGO]
              ,[FECHA_NACIMIENTO]
              ,[IDENTIFICADOR]
              ,[COD_PAIS]
              ,[PROFESION]
              ,[EMPLEADOR]
              ,[CARGO]
              ,[GIRO]
              ,[TELEFONO]
              ,[CELULAR]
              ,[EMAIL]
              ,[DSC_PERFIL_RIESGO]
          FROM [Capitaria].[dbo].[CLIENTES_JG]

        WHERE 
        [IDENTIFICADOR] IN ({identificadores_str}); 
        
        """


}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_APORET_detalle = pd.read_sql(query_universal['query_APORET'], conn)
df_DATOS_clientes = pd.read_sql(query_universal['query_DATOS'], conn)


# CLOSE
conn.close()

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_11384\635906783.py:71: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_APORET_detalle = pd.read_sql(query_universal['query_APORET'], conn)
C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_11384\635906783.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_DATOS_clientes = pd.read_sql(query_universal['query_DATOS'], conn)


In [9]:
query_universal = {
    
        'query_CUSTODIA': f"""
        SELECT
            C.fecha_cierre,
            C.PRECIO_TASA_MERCADO,
            V.NUM_CUENTA,
            V.DSC_CUENTA AS NOMBRE_CLI,
            C.CANTIDAD AS LIBRE,
            CASE 
                WHEN PRESTAMOS IS NULL THEN 0 
                ELSE PRESTAMOS 
            END AS PRESTAMOS_ACC,
            V.IDENTIFICADOR,
            I.NEMOTECNICO,
            V.NOMBRE_ASESOR,
            CASE 
                WHEN COD_SUB_CLASE_INSTRUMENTO IN ('CFI', 'ACC') AND I.COD_MONEDA = 'CLP' THEN CANTIDAD * PRECIO_TASA_MERCADO 
                WHEN COD_SUB_CLASE_INSTRUMENTO IN ('CFI', 'ACC') AND I.COD_MONEDA <> 'CLP' THEN (VALOR_MERCADO_CLP / (CANTIDAD + GARANTIA) * CANTIDAD)
                ELSE (VALOR_MERCADO_CLP / (CANTIDAD + GARANTIA) * CANTIDAD) 
            END AS LIBRE_CLP,
            CASE 
                WHEN COD_SUB_CLASE_INSTRUMENTO IN ('CFI', 'ACC') AND I.COD_MONEDA = 'CLP' THEN GARANTIA * PRECIO_TASA_MERCADO 
                WHEN COD_SUB_CLASE_INSTRUMENTO IN ('CFI', 'ACC') AND I.COD_MONEDA <> 'CLP' THEN (VALOR_MERCADO_CLP / (CANTIDAD + GARANTIA) * GARANTIA)
                ELSE (VALOR_MERCADO_CLP / (CANTIDAD + GARANTIA) * GARANTIA) 
            END AS GARANTIA_CLP,
            C.COMPRAS_PLAZO * C.PRECIO_TASA_MERCADO AS SIM_COMPRA_CLP,
            C.VENTAS_PLAZO * C.PRECIO_TASA_MERCADO AS SIM_VENTA_CLP,
            C.VALOR_MERCADO_CLP,
            I.COD_SUB_CLASE_INSTRUMENTO
        FROM
            dbo.cierre_cartera_resumida AS C WITH (NOLOCK)
            INNER JOIN dbo.VIEW_CUENTAS AS V WITH (NOLOCK) ON C.ID_CUENTA = V.ID_CUENTA
            INNER JOIN dbo.INSTRUMENTO AS I WITH (NOLOCK) ON C.ID_INSTRUMENTO = I.ID_INSTRUMENTO
        WHERE
            C.FECHA_CIERRE BETWEEN CONVERT(datetime, '{F_ix}', 120) AND CONVERT(datetime, '{F}', 120)
            AND V.IDENTIFICADOR IN ({identificadores_str}) ;
    """
}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_custodia_detalle = pd.read_sql(query_universal['query_CUSTODIA'], conn)


# CLOSE
conn.close()

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_11384\2285896502.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_custodia_detalle = pd.read_sql(query_universal['query_CUSTODIA'], conn)


In [10]:
query_universal = {
    
        'query_SaldoCaja': f"""
    SELECT DISTINCT 
        A.NOMBRE_ASESOR, 
        A.IDENTIFICADOR,
        b.NUM_CUENTA, 
        b.DSC_CUENTA AS NOMBRE_CLI, 
        b.FECHA_CIERRE, 
        b.ESTADO, 
        b.TIPO_CAJA, 
        b.MONTO_MON_CAJA, 
        b.MONTO_EN_PESOS, 
        b.TRANSITO
    FROM 
        dbo.VIEW_CUENTAS AS A
    INNER JOIN (
        SELECT DISTINCT 
            C.ID_CUENTA, 
            C.NUM_CUENTA, 
            C.DSC_CUENTA, 
            A.FECHA_CIERRE, 
            A.EST_SALDO_CAJA AS ESTADO, 
            CASE 
                WHEN DSC_CAJA_CUENTA = 'CAJA PESOS' THEN 'CAJA PESO' 
                ELSE DSC_CAJA_CUENTA 
            END AS TIPO_CAJA, 
            A.MONTO_MON_CAJA, 
            A.MONTO_MON_CUENTA AS MONTO_EN_PESOS, 
            A.MONTO_X_PAGAR_MON_CTA, 
            A.MONTO_X_COBRAR_MON_CTA, 
            A.MONTO_X_COBRAR_MON_CTA - A.MONTO_X_PAGAR_MON_CTA AS TRANSITO
        FROM 
            dbo.CUENTA AS C WITH (NOLOCK)
        INNER JOIN 
            dbo.CAJA_CUENTA AS B WITH (NOLOCK) ON C.ID_CUENTA = B.ID_CUENTA
        INNER JOIN 
            dbo.SALDO_CAJA AS A WITH (NOLOCK) ON B.ID_CAJA_CUENTA = A.ID_CAJA_CUENTA
        WHERE 
            A.FECHA_CIERRE IN (
         '2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05',
		'2024-01-06', '2024-01-07', '2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11', '2024-01-12'
            )
            AND A.EST_SALDO_CAJA = 'VIG' 
            AND (A.MONTO_X_PAGAR_MON_CTA + A.MONTO_X_COBRAR_MON_CTA + A.MONTO_MON_CUENTA) <> 0
    ) AS b ON A.NUM_CUENTA = b.NUM_CUENTA
    WHERE 
        A.IDENTIFICADOR IN ({identificadores_str}); 
""",


}

# CONN TO DB
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=tcp:mi-sql-prd-002.5bf1e89c8cf2.database.windows.net;DATABASE=capitaria;UID=pablobustamante;PWD=S3cur3!PassPB5sta#2024')
# EXECUTE
df_saldo_caja = pd.read_sql(query_universal['query_SaldoCaja'], conn)


# CLOSE
conn.close()      

C:\Users\Francisco Betancour\AppData\Local\Temp\ipykernel_11384\4281302796.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_saldo_caja = pd.read_sql(query_universal['query_SaldoCaja'], conn)


In [17]:
df_saldo_caja.to_excel("Monitoreo Noviembre/Clientes PEP/Caja.xlsx")
df_APORET_detalle.to_excel("Monitoreo Noviembre/Clientes PEP/Aportes.xlsx")
df_DATOS_clientes.to_excel("Monitoreo Noviembre/Clientes PEP/Datos.xlsx")
df_operaciones.to_excel("Monitoreo Noviembre/Clientes PEP/Operaciones.xlsx")
df_custodia_detalle.to_excel("Monitoreo Noviembre/Clientes PEP/Custodia.xlsx")

#### TOMAR DATOS XLSX

In [12]:
df_saldo_caja['IDENTIFICADOR'] = df_saldo_caja['IDENTIFICADOR'].apply(reemplazar_k)
df_custodia_detalle['IDENTIFICADOR'] = df_custodia_detalle['IDENTIFICADOR'].apply(reemplazar_k)
df_operaciones['IDENTIFICADOR'] = df_operaciones['IDENTIFICADOR'].apply(reemplazar_k)
df_DATOS_clientes['IDENTIFICADOR'] = df_DATOS_clientes['IDENTIFICADOR'].apply(reemplazar_k)
df_APORET_detalle['IDENTIFICADOR'] = df_APORET_detalle['IDENTIFICADOR'].apply(reemplazar_k)

### DOLAR OBS V1

In [13]:
df_dolar_obs = pd.read_excel("dol_observado.xlsx") 

### DOLAR OBS V2

In [14]:
# Ensure column names match
df_dolar_obs.rename(columns={'Your_Date_Column': 'FECHA_OPERACION'}, inplace=True)

# Check for missing columns
print(df_operaciones.columns)
print(df_dolar_obs.columns)

# Merge the data
df_operaciones = pd.merge(df_operaciones, df_dolar_obs[['FECHA_OPERACION', 'DOLAR_OBS']], on='FECHA_OPERACION', how='left')

# Replace missing DOLAR_OBS values with 0
if 'DOLAR_OBS' in df_operaciones.columns:
    df_operaciones['DOLAR_OBS'] = pd.to_numeric(df_operaciones['DOLAR_OBS'], errors='coerce').fillna(0)
else:
    print("Error: 'DOLAR_OBS' column is missing after the merge.")

# Ensure MONTO is numeric
df_operaciones['MONTO'] = pd.to_numeric(df_operaciones['MONTO'], errors='coerce')

# Calculate MONTO_CLP
df_operaciones['MONTO_CLP'] = df_operaciones.apply(
    lambda row: row['MONTO'] * row['DOLAR_OBS'] if row['COD_MONEDA_OP'] == 'USD' else row['MONTO'], 
    axis=1
)


Index(['NUM_CUENTA', 'IDENTIFICADOR', 'NOMBRE_CLIENTE', 'NOMBRE_ASESOR',
       'FECHA_OPERACION', 'COD_TIPO_OPERACION', 'DSC_OPERACION_CONCEPTO',
       'COD_MONEDA_OP', 'NEMOTECNICO', 'DSC_INSTRUMENTO',
       'COD_SUB_CLASE_INSTRUMENTO', 'CANTIDAD', 'MONTO', 'MONTO_OPERACION'],
      dtype='object')
Index(['FECHA_OPERACION', 'DOLAR_OBS'], dtype='object')


In [15]:
%store df_muestra2
%store df_saldo_caja
%store df_custodia_detalle
%store df_operaciones
%store df_DATOS_clientes
%store df_APORET_detalle
# cantidad de clientes en la muestra
%store m

Stored 'df_muestra2' (DataFrame)
Stored 'df_saldo_caja' (DataFrame)
Stored 'df_custodia_detalle' (DataFrame)
Stored 'df_operaciones' (DataFrame)
Stored 'df_DATOS_clientes' (DataFrame)
Stored 'df_APORET_detalle' (DataFrame)
Stored 'm' (int)


Check

In [16]:
df_operaciones[df_operaciones['IDENTIFICADOR'] == '16322686-3'].sort_values(by='FECHA_OPERACION')

,NUM_CUENTA,IDENTIFICADOR,NOMBRE_CLIENTE,NOMBRE_ASESOR,FECHA_OPERACION,COD_TIPO_OPERACION,DSC_OPERACION_CONCEPTO,COD_MONEDA_OP,NEMOTECNICO,DSC_INSTRUMENTO,COD_SUB_CLASE_INSTRUMENTO,CANTIDAD,MONTO,MONTO_OPERACION,DOLAR_OBS,MONTO_CLP
